In [7]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [8]:
data = pd.read_csv('../cgarrido/berlin_data/weather/berlin_weather_2022.csv', sep=',')

In [9]:
data

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2022-01-01,11.4,10.5,12.0,1.3,NaN,254.0,16.7,37.0,1021.4,NaN
1,2022-01-02,10.2,9.1,11.3,2.1,NaN,218.0,18.2,44.5,1012.9,NaN
2,2022-01-03,9.1,7.5,10.2,3.8,NaN,248.0,24.1,51.8,1006.5,NaN
3,2022-01-04,7.2,5.5,8.2,2.9,NaN,231.0,12.1,27.8,996.7,NaN
4,2022-01-05,3.9,3.2,4.9,1.7,NaN,251.0,21.6,48.2,997.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,4.4,3.0,6.0,0.0,NaN,249.0,21.6,48.2,1020.9,NaN
361,2022-12-28,6.0,3.2,8.1,2.1,NaN,203.0,19.2,37.0,1014.2,NaN
362,2022-12-29,9.5,7.7,11.4,1.3,NaN,211.0,21.6,42.6,1005.6,NaN
363,2022-12-30,7.6,6.4,9.0,1.5,NaN,206.0,19.6,40.8,1010.9,NaN


In [15]:
entity_id_prefix =  "urn:ngsi-ld:Weather:Berlin"
entity_obj = Entity('WeatherObserved', entity_id_prefix ) # (re-)initialize the entity
fac = True
for indx, itm in data.iterrows():
    entity_obj.add_prop("tavg", str(itm['tavg']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("tmin", str(itm['tmin']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("tmax", str(itm['tmax']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("prcp", str(itm['prcp']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("wdir", str(itm['wdir']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("wspd", str(itm['wspd']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("wpgt", str(itm['wpgt']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    entity_obj.add_prop("pres", str(itm['pres']) , observedAt= datetime.strptime(str(itm.date), '%Y-%m-%d'))
    if fac == True :
        print(entity_obj)
publish(entity_obj.to_dict())




{'id': 'urn:ngsi-ld:Weather:Berlin', 'type': 'WeatherObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'tavg': {'type': 'Property', 'value': '11.4', 'observedAt': '2022-01-01T00:00:00Z'}, 'tmin': {'type': 'Property', 'value': '10.5', 'observedAt': '2022-01-01T00:00:00Z'}, 'tmax': {'type': 'Property', 'value': '12.0', 'observedAt': '2022-01-01T00:00:00Z'}, 'prcp': {'type': 'Property', 'value': '1.3', 'observedAt': '2022-01-01T00:00:00Z'}, 'wdir': {'type': 'Property', 'value': '254.0', 'observedAt': '2022-01-01T00:00:00Z'}, 'wspd': {'type': 'Property', 'value': '16.7', 'observedAt': '2022-01-01T00:00:00Z'}, 'wpgt': {'type': 'Property', 'value': '37.0', 'observedAt': '2022-01-01T00:00:00Z'}, 'pres': {'type': 'Property', 'value': '1021.4', 'observedAt': '2022-01-01T00:00:00Z'}}
{'id': 'urn:ngsi-ld:Weather:Berlin', 'type': 'WeatherObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'tavg': {'type': 'Property', 'value'

{'status_code': 204}

In [ ]:




example_flag=False
for property_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    prop =gdf_property_list[property_index]

    hexagon_id_list = tmp_gdf['hexagon_id'].unique()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]
        
        entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity

        counter =0
        for index, row in hexagon_gdf.iterrows():
            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lat, lon))
                    entity_obj.add_geoprop("location", location_value, observedAt= datetime.strptime(str(row.datetime), '%d.%m.%Y %H:%M')) # MAKE THIS A POINT (SIVA EXAMPLE)
                elif feature_list[f] in ["co", "pm10", "pm2.5", "no2", "o3"]:
                    if(feature_list[f] == "pm2.5"):
                        print(feature_list[f])
                        entity_obj.add_prop("pm2_5", str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%d.%m.%Y %H:%M'))                 
                    else :
                        entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%d.%m.%Y %H:%M'))
                #elif(feature_list[f] not in ['datetime','type']):
                #    entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully' 
        
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))
